In [1]:
import pandas as pd
import numpy as np
import filtering

데이터 불러오기

In [2]:
raw_rating_data= pd.read_csv('./data/rating.csv')

In [3]:
raw_movie_data= pd.read_csv('./data/movie.csv')

필터링

In [18]:
# 필터링 
rating_threshold = 500
genres_included = ['Action', 'Comedy', 'Drama']
movie_min_year = 1995

In [19]:
movie_data = filtering.movie_filter(raw_movie_data, movie_min_year, genres_included)

In [20]:
rating_data = filtering.rating_filter(raw_rating_data, movie_data, rating_threshold) 

In [21]:
len(raw_rating_data)

20000263

In [22]:
len(rating_data)

3156465

In [23]:
len(raw_movie_data)

27278

In [24]:
len(movie_data)

11574

은선님 코드

In [25]:
from scipy.sparse.linalg import svds

In [41]:
def make_prediction_df(rating_data):
    R_df = rating_data.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

    #R은 pivot_table을 sparse matrix로 만든 것
    R = R_df.values
    
    #user_rating_mean은 사용자의 평균 평점
    user_ratings_mean = np.mean(R, axis = 1)
    
    #R_demeaned : 사용자-영화 테이블에 대해 사용자 평균 평점을 뺀 것
    R_demeaned = R - user_ratings_mean.reshape(-1, 1)
        
    #U 행렬, sigma 행렬, V전치행렬을 반환.
    #이때 spicy에 있는 svd를 이용한다.
    U, sigma, Vt = svds(R_demeaned, k = 50)
    
    #sigma는 0이 포함되지 않은 값으로만 구성되어 있다.
    #sigma를 0이 포함된 대칭 행렬로 변환한다.
    sigma = np.diag(sigma)
    
    #SVD가 적용되어 분해된 R_demeaned를 원본 행렬로 복구
    #구한 원본 행렬에 사용자 평균 rating을 더해준다.
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)
    
    preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)
    
    return preds_df


In [43]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    #인덱스로 변환해주어야 해서 -1 
    user_row_number = userID - 1
    
    #앞에서 만든 prediction행렬을 사용자 인덱스에 따라 영화 정렬 -> 영화 평점이 높은 순으로 정렬
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    #원본 rating_data에서 user_id에 해당하는 데이터를 뽑아낸다.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    
    #user_data와 원본 영화 데이터를 합친다.
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId').
                     sort_values(['rating'], ascending=False)
                 )
    
    #원본 영화 데이터에서 사용자가 본 영화를 제외한 데이터 추출
    #.merge로 sorted_user_predcitions와 user_full데이터를 합친다.
    #.rename으로 컬럼 이름을 바꾸고 정렬한다
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False)
                      )
                       
    return recommendations


In [44]:
#make_prediction_df()함수 호출하기 전에 input을 넣어줍니다.
preds_df = make_prediction_df(rating_data)

In [45]:
#24번 사용자가 본 영화 데이터를 토대로 영화 추천 받은 목록
prediction= recommend_movies(preds_df, 24, movie_data, rating_data, 10)

In [46]:
prediction

,movieId,title,genres,year,Predictions
845,2395,Rushmore (1998),"[Comedy, Drama]",1998.0,6.086091
1513,4979,"Royal Tenenbaums, The (2001)","[Comedy, Drama]",2001.0,4.888511
1248,3911,Best in Show (2000),[Comedy],2000.0,4.878647
923,2599,Election (1999),[Comedy],1999.0,4.667829
456,1449,Waiting for Guffman (1996),[Comedy],1996.0,4.360885
3217,33794,Batman Begins (2005),"[Action, Crime, IMAX]",2005.0,4.226172
267,745,Wallace & Gromit: A Close Shave (1995),"[Animation, Children, Comedy]",1995.0,4.199637
3401,39292,"Good Night, and Good Luck. (2005)","[Crime, Drama]",2005.0,4.172835
3729,46578,Little Miss Sunshine (2006),"[Adventure, Comedy, Drama]",2006.0,4.057806
2575,8961,"Incredibles, The (2004)","[Action, Adventure, Animation, Children, Comedy]",2004.0,3.979754


MemoryError: 

In [ ]:
prediction

In [ ]:
df_pivot.values

In [ ]:
user_ratings_mean = np.mean(df_pivot, axis = 1)

In [ ]:
user_ratings_mean

In [ ]:
preds_df.head()